# Database Privileges Test Notebook

This notebook tests whether your database user has the necessary privileges to alter tables without actually making any changes.

In [5]:
import sys
import os
import psycopg2
from psycopg2 import sql

# Add project root to path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)

from iris.src.initial_setup.db_config import get_db_params, connect_to_db

## 1. Connect to Database

In [6]:
# Get database parameters
db_params = get_db_params("local")
print(f"Connecting to {db_params['dbname']} at {db_params['host']}:{db_params['port']} as user {db_params['user']}...")

# Connect to database
conn = connect_to_db("local")
if conn:
    print("Connected successfully! 🎉")
else:
    print("Failed to connect to database.")

Connecting to maven-finance at localhost:5432 as user iris_dev...
Connected successfully! 🎉


## 2. Check User Privileges

In [7]:
def check_user_privileges(conn):
    """Check the current user's privileges on database objects."""
    with conn.cursor() as cur:
        # Get current user
        cur.execute("SELECT current_user;")
        current_user = cur.fetchone()[0]
        print(f"Current database user: {current_user}")
        
        # Check table privileges
        cur.execute("""
            SELECT table_name, privilege_type
            FROM information_schema.table_privileges
            WHERE grantee = current_user
              AND table_schema = 'public'
              AND table_name IN ('apg_catalog', 'apg_content')
            ORDER BY table_name, privilege_type;
        """)
        
        privileges = cur.fetchall()
        if privileges:
            print("\nYour privileges on the tables:")
            for table_name, privilege in privileges:
                print(f"- {table_name}: {privilege}")
        else:
            print("\nNo explicit privileges found for your user on the target tables.")
            print("This might be because privileges are granted through a role or group.")
        
        # Check if user is a superuser
        cur.execute("""SELECT usesuper FROM pg_user WHERE usename = current_user;""")
        is_superuser = cur.fetchone()[0]
        if is_superuser:
            print("\n✅ You are a superuser, which means you have all privileges.")
        else:
            print("\nYou are not a superuser.")

if conn:
    check_user_privileges(conn)

Current database user: iris_dev

No explicit privileges found for your user on the target tables.
This might be because privileges are granted through a role or group.

✅ You are a superuser, which means you have all privileges.


## 3. Test ALTER TABLE Permissions (with Rollback)

In [8]:
def test_alter_table_permission(conn):
    """Test ALTER TABLE permission using a transaction that will be rolled back."""
    if not conn:
        return
    
    print("Testing ALTER TABLE permission...")
    print("NOTE: No actual changes will be made - we'll roll back the transaction.")
    
    # Create a new connection for this test to avoid transaction issues
    test_conn = None
    try:
        # Connect with a fresh connection
        db_params = get_db_params("local")
        test_conn = psycopg2.connect(**db_params)
        test_conn.autocommit = False  # Start in transaction mode
        
        with test_conn.cursor() as cur:
            # Try to add a test column to apg_catalog
            cur.execute("""
                ALTER TABLE apg_catalog 
                ADD COLUMN _test_permission_column TEXT;
            """)
            print("\n✅ ALTER TABLE permission test successful! You have permission to alter tables.")
    except Exception as e:
        print(f"\n❌ ALTER TABLE permission test failed: {str(e)}")
        print("You may not have permission to alter tables.")
    finally:
        # Roll back the transaction - no actual changes are made
        if test_conn:
            test_conn.rollback()
            print("\nTransaction rolled back - no changes were made to the database.")
            test_conn.close()

if conn:
    test_alter_table_permission(conn)

Testing ALTER TABLE permission...
NOTE: No actual changes will be made - we'll roll back the transaction.

✅ ALTER TABLE permission test successful! You have permission to alter tables.

Transaction rolled back - no changes were made to the database.


## 4. Check for pgvector Extension

In [9]:
def check_pgvector_extension(conn):
    """Check if the pgvector extension is available and installed."""
    if not conn:
        return
    
    print("Checking for pgvector extension...")
    
    with conn.cursor() as cur:
        # Check if pgvector is available
        cur.execute("""
            SELECT name, default_version, installed_version
            FROM pg_available_extensions
            WHERE name = 'vector';
        """)
        
        result = cur.fetchone()
        if result:
            name, default_version, installed_version = result
            print(f"\n✅ pgvector extension is available:")
            print(f"   - Name: {name}")
            print(f"   - Default version: {default_version}")
            
            if installed_version:
                print(f"   - Installed version: {installed_version}")
                print("   - Status: INSTALLED")
            else:
                print("   - Status: AVAILABLE BUT NOT INSTALLED")
                print("   - You'll need to install it with: CREATE EXTENSION vector;")
                
            # Test if you have permission to create extensions
            # Create a new connection for this test to avoid transaction issues
            test_conn = None
            try:
                # Connect with a fresh connection
                db_params = get_db_params("local")
                test_conn = psycopg2.connect(**db_params)
                test_conn.autocommit = False  # Start in transaction mode
                
                with test_conn.cursor() as test_cur:
                    # Try to create the extension
                    test_cur.execute("""CREATE EXTENSION IF NOT EXISTS vector;""")
                    print("\n✅ You have permission to create/manage extensions.")
            except Exception as e:
                print(f"\n❌ Extension creation test failed: {str(e)}")
                print("You may not have permission to create extensions.")
            finally:
                # Roll back the transaction - no actual changes are made
                if test_conn:
                    test_conn.rollback()
                    print("Transaction rolled back - no changes were made to the database.")
                    test_conn.close()
        else:
            print("\n❌ pgvector extension is not available on this PostgreSQL server.")
            print("You'll need to install the pgvector extension on the server before using vector columns.")

if conn:
    check_pgvector_extension(conn)

Checking for pgvector extension...

❌ pgvector extension is not available on this PostgreSQL server.
You'll need to install the pgvector extension on the server before using vector columns.


## 5. Test Vector Column Creation (with Rollback)

In [10]:
def test_vector_column_creation(conn):
    """Test if we can create a vector column (with rollback)."""
    if not conn:
        return
    
    print("Testing vector column creation...")
    print("NOTE: No actual changes will be made - we'll roll back the transaction.")
    
    # First check if vector extension is installed
    with conn.cursor() as cur:
        cur.execute("""
            SELECT installed_version FROM pg_available_extensions WHERE name = 'vector';
        """)
        result = cur.fetchone()
    
    if not result or not result[0]:
        print("\n❓ Cannot test vector column creation because the vector extension is not installed.")
        print("You'll need to install the extension first with: CREATE EXTENSION vector;")
        return
    
    # Create a new connection for this test to avoid transaction issues
    test_conn = None
    try:
        # Connect with a fresh connection
        db_params = get_db_params("local")
        test_conn = psycopg2.connect(**db_params)
        test_conn.autocommit = False  # Start in transaction mode
        
        with test_conn.cursor() as cur:
            # Try to add a vector column
            cur.execute("""
                ALTER TABLE apg_catalog 
                ADD COLUMN _test_vector_column vector(2000);
            """)
            print("\n✅ Vector column creation test successful!")
            print("You can create vector columns in your database.")
    except Exception as e:
        print(f"\n❌ Vector column creation test failed: {str(e)}")
    finally:
        # Roll back the transaction - no actual changes are made
        if test_conn:
            test_conn.rollback()
            print("\nTransaction rolled back - no changes were made to the database.")
            test_conn.close()

if conn:
    test_vector_column_creation(conn)

Testing vector column creation...
NOTE: No actual changes will be made - we'll roll back the transaction.

❓ Cannot test vector column creation because the vector extension is not installed.
You'll need to install the extension first with: CREATE EXTENSION vector;


## 6. Summary

In [11]:
print("\n=== SUMMARY ===\n")
print("This notebook has tested your database permissions without making any actual changes.")
print("Review the output above to determine if you have the necessary permissions to:")
print("1. Alter tables")
print("2. Create/manage extensions (if needed for pgvector)")
print("3. Create vector columns (if pgvector is installed)")

# Close the connection
if conn:
    conn.close()
    print("\nDatabase connection closed.")


=== SUMMARY ===

This notebook has tested your database permissions without making any actual changes.
Review the output above to determine if you have the necessary permissions to:
1. Alter tables
2. Create/manage extensions (if needed for pgvector)
3. Create vector columns (if pgvector is installed)

Database connection closed.
